# CRS Index

By default rioxarray stores CRS information as a scalar coordinate in Xarray. This works well because the important metadata persists through all Xarray operations and when it comes time to save an output, we must retrieve this important metadata.

As of Xarray v2022.09.0 it is possible to create custom Indexes (typically a `PandasIndex` is used behind the scenes that determines how dimensional coordinates are queried and aligned). With the new Xindexes interface, we can attach CRS as persistent metadata to a custom `CRSIndex`. Effectively we enhance the default `PandasIndex` by adding associated metadata (CRS) and new functionality like checking CRS compatibility across objects.

This notebook showcases experimental CRSIndex functionality

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr

xr.set_options(
    display_expand_data=False,
    display_expand_indexes=True,
)


In [4]:
rds = xr.open_dataset("../../test/test_data/input/PLANET_SCOPE_3D.nc", engine="rasterio")

IN WRITE_CRS
x
IN WRITE_CRS
x


ValueError: those coordinates don't exist: {'y', 'x'}

In [18]:
rds

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
    spatial_ref  int64 ...
Data variables:
    blue         (time, y, x) float64 ...
    green        (time, y, x) float64 ...
Attributes:
    coordinates:  spatial_ref

Note `x` and `y` dimensions are backed by a corresponding separate `PandasIndex` by default. `spatial_ref` is a non-dimensional scalar coordinate and therefore does not appear in the `Indexes`

In [19]:
# CRS information is retrieved via the `.rio` accessor
rds.rio.crs

CRS.from_epsg(32722)

In [20]:
# Convert From PandasIndex to CRSIndex
rds = rds.rio.write_crs(use_crs_index=True)
rds

IN WRITE_CRS


<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
Data variables:
    blue         (time, y, x) float64 ...
    green        (time, y, x) float64 ...
Indexes:
    x            CRSIndex (crs=EPSG:32722)
    y            CRSIndex (crs=EPSG:32722)
Attributes:
    coordinates:  spatial_ref

Now the spatial relationship between `x` and `y` is indicated by their shared `CRSIndex`, which includes an abbreviated representation (often an EPSG code)

## Using CRSIndex

In [23]:
rdsll = rds.rio.reproject('EPSG:4326')
rdsll

IN WRITE_CRS
IN WRITE_CRS


<xarray.Dataset>
Dimensions:      (x: 10, y: 10, time: 2)
Coordinates:
  * x            (x) float64 -51.32 -51.32 -51.32 ... -51.32 -51.32 -51.32
  * y            (y) float64 -17.32 -17.32 -17.32 ... -17.32 -17.32 -17.32
  * time         (time) object 2016-12-19 10:27:29.687763 2016-12-29 12:52:42...
    spatial_ref  int64 0
Data variables:
    blue         (time, y, x) float64 6.611 5.581 0.3996 ... 3.491 5.056 3.368
    green        (time, y, x) float64 7.921 66.15 30.1 ... 21.76 27.29 18.41
Attributes:
    coordinates:  spatial_ref